In [406]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [407]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import KFold

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [408]:
TRAIN_DATASET_PATH = '../input/realestate-price-prediction-moskow/train.csv'
TEST_DATASET_PATH = '../input/realestate-price-prediction-moskow/test.csv'

Id - идентификационный номер квартиры

DistrictId - идентификационный номер района

Rooms - количество комнат

Square - площадь

LifeSquare - жилая площадь

KitchenSquare - площадь кухни

Floor - этаж

HouseFloor - количество этажей в доме

HouseYear - год постройки дома

Ecology_1, Ecology_2, Ecology_3 - экологические показатели местности

Social_1, Social_2, Social_3 - социальные показатели местности

Healthcare_1, Helthcare_2 - показатели местности, связанные с охраной здоровья

Shops_1, Shops_2 - показатели, связанные с наличием магазинов, торговых центров

Price - цена квартиры

In [409]:
train = pd.read_csv(TRAIN_DATASET_PATH)
train.head()

In [410]:
test = pd.read_csv(TEST_DATASET_PATH)
test.head()

In [411]:
train.dtypes

In [412]:
train.describe().transpose()

In [413]:
train.info()

In [414]:
train.isna().sum()

Посмотрим зависимость жилой площади от общей

In [415]:
grid = sns.jointplot(train['Square'], train['LifeSquare'], kind='reg')
grid.fig.set_figwidth(8)
grid.fig.set_figheight(8)

plt.show()

Посмотрим зависимость цены от общей площади 

In [416]:
grid = sns.jointplot(train['Price'], train['Square'], kind='reg')
grid.fig.set_figwidth(8)
grid.fig.set_figheight(8)

plt.show()

Подготовка данных

In [417]:
def data_preprocessing(df):
    
    # приводим тип float к int
    
    df['Rooms'] = df['Rooms'].astype('int64')
    df.replace({'Rooms':{0: df['Rooms'].mode()[0]}}, inplace=True)
    
    df['HouseFloor'] = df['HouseFloor'].astype('int64')
    df.replace({'HouseFloor':{0: df['HouseFloor'].mode()[0]}}, inplace=True)
    
    # Заменяем Nan на средние значения
    
    mean_perc_rel = ((df['LifeSquare']/df['Square'])).mean()
    df['LifeSquare'] = df['LifeSquare'].fillna(df['Square'] * mean_perc_rel)
    
    median = df['Healthcare_1'].median()
    df['Healthcare_1'] = df['Healthcare_1'].fillna(median)
    
    # Заменяем значения категориальных признаков на цифры и приводим тип к int
    
    df.replace({'Ecology_2': {'A': '1'}},inplace=True)
    df.replace({'Ecology_2': {'B': '2'}},inplace=True)
    df['Ecology_2'].astype('int64')
    
    df.replace({'Ecology_3': {'A': '1'}},inplace=True)
    df.replace({'Ecology_3': {'B': '2'}},inplace=True)
    df['Ecology_3'].astype('int64')
    
    df.replace({'Shops_2': {'A': '1'}},inplace=True)
    df.replace({'Shops_2': {'B': '2'}},inplace=True)
    df['Shops_2'].astype('int64')
    
    #  Заполняем нулевые значения средними, устраняем выбросы
    
    df.replace({'KitchenSquare':{0: df['KitchenSquare'].mode()[0]}}, inplace=True)
    kitchen_square_maxval = np.quantile(df['KitchenSquare'], q=0.975)
    kitchen_square_minval = np.quantile(df['KitchenSquare'], q=0.025)
    condition = (df['KitchenSquare'] > kitchen_square_maxval) | (df['KitchenSquare'] < kitchen_square_minval)
    df.loc[condition, 'KitchenSquare'] = df['KitchenSquare'].median()
    
    houseyear_maxval = np.quantile(df['HouseYear'], q=0.975)
    houseyear_minval = np.quantile(df['HouseYear'], q=0.025)
    condition = (df['HouseYear'] > houseyear_maxval) | (df['HouseYear'] < houseyear_minval)
    df.loc[condition, 'HouseYear'] = df['HouseYear'].median()
    
    square_maxval = np.quantile(df['Square'], q=0.975)
    square_minval = np.quantile(df['Square'], q=0.025)
    condition = (df['Square'] > square_maxval) | (df['Square'] < square_minval)
    df.loc[condition, 'Square'] = df['Square'].median()
    
    life_square_maxval = np.quantile(df['LifeSquare'], q=0.975)
    life_square_minval = np.quantile(df['LifeSquare'], q=0.025)
    condition = (df['LifeSquare'] > life_square_maxval) | (df['LifeSquare'] < life_square_minval)
    df.loc[condition, 'LifeSquare'] = df['LifeSquare'].median()
    

In [418]:
data_preprocessing(train)

In [419]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [420]:
reduce_mem_usage(train)


Построим график распределения цены

In [421]:
target_mean = round(train['Price'].mean(), 2)
target_median = train['Price'].median()
target_mode = train['Price'].mode()[0]

plt.figure(figsize = (16, 8))

sns.distplot(train['Price'], bins=50)

y = np.linspace(0, 0.000008, 10)
plt.plot([target_mean] * 10, y, label='mean', linestyle=':', linewidth=4)
plt.plot([target_median] * 10, y, label='median', linestyle='--', linewidth=4)
plt.plot([target_mode] * 10, y, label='mode', linestyle='-.', linewidth=4)

plt.title('Distribution of Price')
plt.legend()
plt.show()

In [422]:
train.info()

Разбиваем датасет на тренировочный и тестовый

In [423]:
features = ['DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear',
                 'Ecology_1', 'Ecology_2', 'Ecology_3', 'Social_1', 'Social_2', 'Social_3', 'Healthcare_1'
                 'Helthcare_2', 'Shops_1', 'Shops_2']
target_name = 'Price'

X = train.drop(columns=target_name)
y = train[target_name]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4, shuffle=True, random_state=42)

In [424]:
X_train.shape, X_valid.shape, test.shape

Создаем модель

In [425]:
from sklearn.ensemble import GradientBoostingRegressor

gb_model = GradientBoostingRegressor(criterion='friedman_mse',
                                     max_depth=9,
                                     min_samples_leaf=10,
                                     random_state=42,  
                                     n_estimators=25)
gb_model.fit(X_train, y_train)


In [426]:
def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):
    print("Train R2:\t" + str(round(r2(train_true_values, train_pred_values), 3)))
    print("Test R2:\t" + str(round(r2(test_true_values, test_pred_values), 3)))
    
    plt.figure(figsize=(18,10))
    
    plt.subplot(121)
    sns.scatterplot(x=train_pred_values, y=train_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Train sample prediction')
    
    plt.subplot(122)
    sns.scatterplot(x=test_pred_values, y=test_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Test sample prediction')

    plt.show()

Оценка предсказаний цены

In [427]:
y_train_preds = gb_model.predict(X_train)
y_test_preds = gb_model.predict(X_valid)

evaluate_preds(y_train, y_train_preds, y_valid, y_test_preds)

In [428]:
cv_score = cross_val_score(gb_model, X_train, y_train, scoring='r2', cv=KFold(n_splits=3, shuffle=True, random_state=42))
cv_score

In [429]:
cv_score.mean()

In [430]:
feature_importances = pd.DataFrame(zip(X_train.columns, gb_model.feature_importances_), 
                                   columns=['feature', 'importance'])

feature_importances.sort_values(by='importance', ascending=False)

Подготовка тестового датасета

In [431]:
data_preprocessing(test)
reduce_mem_usage(test)

In [432]:
test['Price'] = gb_model.predict(test) 

test[['Id', 'Price']].head()

In [433]:
test[['Id', 'Price']].to_csv('juliaconstanto_price_predict.csv', index=None)